<div style="float: right; margin-left: 20px;">
  <img src="logo.png" alt="Logo" width="105" height="105"/>
</div>

Democratic and Popular Republic of Algeria \
Ministry of Higher Education and Scientific Research \
National Higher School of Computer Science - May 9 1945 - Sidi Bel Abbes \
### Multi-Agent Systems
### TP 03: Game-Based Metaheuristics 
___
#### Table de Matiéres 
1. [Data Preprocessing](#data-preprocessing)
2. [Machine Learning Classification](#ml)
3. [Classifier ojective function](#classifier)
4. [Tug of War function](#tow)
5. [Hide Object Game optimizer](#hogo)
6. [League Championship algorithm](#lca)
7. [Most Valuable Player algorithm](#mvpa)
8. [Testing](#test)
9. [Evaluation](#eval)
10. [Ludo Game](#ludo)

For more information about the dataset, click [here](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets/data).

By Imene Bouaziz G01 ISI

<hr style="border: 0.3px solid black;"/>

## Importing Libraries

In [2]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.metrics import classification_report

## <span style="color:blue;">1. Data Preprocessing</span> <a class="anchor" id="data-preprocessing"></a>

### 1.1. Loading the dataset

In [3]:
true_data=pd.read_csv('./News _dataset/True.csv')
fake_data=pd.read_csv('./News _dataset/Fake.csv')

print(true_data[0:10])
fake_data[0:4]

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   
5  White House, Congress prepare for talks on spe...   
6  Trump says Russia probe will be fair, but time...   
7  Factbox: Trump on Twitter (Dec 29) - Approval ...   
8         Trump on Twitter (Dec 28) - Global Warming   
9  Alabama official to certify Senator-elect Jone...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) -

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"


### 1.2. Dimention

In [3]:
len(true_data)

21417

In [4]:
len(fake_data)

23481

### 1.3. Labet attribution

In [5]:
fake_data['label'] = 1
true_data['label']=0
print(true_data.head())
fake_data.head()

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  label  
0  December 31, 2017       0  
1  December 29, 2017       0  
2  December 31, 2017       0  
3  December 30, 2017       0  
4  December 29, 2017       0  


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


### 1.4.Concatination and shuffling 

In [6]:
all_data=pd.concat([fake_data, true_data])
random_permutation = np.random.permutation(len(all_data))
all_data= all_data.iloc[random_permutation]
print(all_data.columns)
#all_data.head()

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')


### 1.5. Feature selection

In [7]:
filterd_data=all_data.loc[:, ['title', 'text', "subject", 'label']]
#filterd_data.head()

### 1.6. Missing values

In [8]:
filterd_data.isnull().sum()

title      0
text       0
subject    0
label      0
dtype: int64

### 1.7. Feature engineering

In [9]:
filterd_data['training_feature']=filterd_data['title']+' '+filterd_data['text']+' '+filterd_data['subject']
#filterd_data.head()

### 1.8. Creating data subsets

In [10]:
X= filterd_data['training_feature'].values
y = filterd_data['label']

In [11]:
l_X=filterd_data['training_feature'].values[0:1000]
l_Y= filterd_data['label'].values[0:1000]

In [12]:
print(l_X.shape)
print(l_Y.shape)
type (l_X)

(1000,)
(1000,)


numpy.ndarray

### 1.9. Converting to matrix  

In [13]:
vectorizer= TfidfVectorizer()
X=vectorizer.fit_transform(X)

In [14]:
l_vectorizer= TfidfVectorizer()
l_X=l_vectorizer.fit_transform(l_X)

In [15]:
print(type(X))
print(X.shape)
print(type(l_X))
print(l_X.shape)

<class 'scipy.sparse._csr.csr_matrix'>
(44898, 122516)
<class 'scipy.sparse._csr.csr_matrix'>
(1000, 22489)


### 1.10. Splitting the dataset

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [17]:
l_X_train, l_X_test, l_Y_train, l_Y_test = train_test_split(l_X, l_Y, test_size = 0.2, random_state=42)

In [18]:
X_train.shape

(35918, 122516)

## <span style="color:blue;">2. Machine learning classification </span> <a class="anchor" id="ml"></a>

In [19]:
#Create a svm Classifier
model = svm.SVC(kernel='linear') # Linear Kernel
clf_poly=clf = svm.SVC(kernel='poly') # Polynomial kernel

In [20]:
print(X_train.shape)
# X_train.iloc[0:1000].shape

(35918, 122516)


In [21]:
#Train the model using the training sets
model.fit(l_X_train, l_Y_train)
# clf_poly.fit(X_train,Y_train)

SVC(kernel='linear')

In [22]:
#Predict the response for test dataset
y_pred = model.predict(l_X_test)

In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print(accuracy_score(y_pred, l_Y_test))
print (precision_score(l_Y_test, y_pred,))
print (recall_score(l_Y_test, y_pred,))
print(f1_score(l_Y_test, y_pred,))
print(confusion_matrix(l_Y_test, y_pred,))
print(classification_report(l_Y_test, y_pred))

0.975
0.9896907216494846
0.96
0.9746192893401014
[[99  1]
 [ 4 96]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       100
           1       0.99      0.96      0.97       100

    accuracy                           0.97       200
   macro avg       0.98      0.97      0.97       200
weighted avg       0.98      0.97      0.97       200



## <span style="color:blue;">3. Classifier ojective function</span> <a class="anchor" id="classifier"></a>

In [126]:
from sklearn.svm import SVC
# Define the objective function to be optimized
def objective_function(x):
    #apply this one to execute mvpa and comment it to use the others
    svm = SVC(C=x["C"], gamma=x["gamma"], kernel='rbf')
    #apply this one to execute two and hogo and comment it to use mvpa
    #svm = SVC(C=x[0], gamma=x[1], kernel='rbf')
    svm.fit(l_X_train, l_Y_train)
    y_pred = svm.predict(l_X_test)
    # Assuming binary classification, you can use F1 score as the objective function
    accuracy = accuracy_score(l_Y_test, y_pred)
   # f1 = f1_score(l_Y_test, y_pred)
    # Return the negative F1 score (to be minimized by TWO)
    return accuracy

## <span style="color:blue;">4. Tug of War function</span> <a class="anchor" id="tow"></a>

In [27]:
def tug_of_war_optimization(objective_function, n_iterations, n_teams, n_members, lb, ub):
    # Initialize the population of teams
    teams = np.random.uniform(lb, ub, size=(n_teams, n_members))

    # Evaluate the initial population
    fitness = np.array([objective_function(team) for team in teams])

    # Iterate over the specified number of iterations
    for i in range(n_iterations):
        # Compute the pulling forces between teams
        forces = np.zeros((n_teams, n_members))
        for j in range(n_teams):
            for k in range(n_teams):
                if j != k:
                    diff = teams[k] - teams[j]
                    dist = np.linalg.norm(diff)
                    force = (fitness[k] - fitness[j]) * diff / dist
                    forces[j] += force
                    forces[k] -= force

        # Update the positions of the teams based on the pulling forces
        velocities = forces / n_members
        teams += velocities
        teams = np.clip(teams, lb, ub)

        # Evaluate the new population
        new_fitness = np.array([objective_function(team) for team in teams])

        # Update the fitness values and best solution
        idx = new_fitness < fitness
        fitness[idx] = new_fitness[idx]
        
        # Print the progress
        print(f"Iteration {i + 1}: Fitness values: {fitness}")

    # Return the best solution and fitness value
    best_idx = np.argmax(fitness)
    best_solution_two = tuple(teams[best_idx])
    best_fitness_two = fitness[best_idx]

    return best_solution_two, best_fitness_two

## <span style="color:blue;">5. Hide Object Game optimizer</span> <a class="anchor" id="hogo"></a>

In [28]:
def hogo(objective_function, n_iterations, n_agents, n_dimensions, lb, ub, verbose=False):
    # Initialize the agents randomly within the search space
    agents = np.random.uniform(lb, ub, size=(n_agents, n_dimensions))
    old_agents = agents.copy()

    # Evaluate the initial population
    fitness = np.array([objective_function(agent) for agent in agents])

    # Iterate over the specified number of iterations
    for i in range(n_iterations):
        # Select the best and worst agents as the "hidden objects"
        best_agent_idx = np.argmax(fitness)
        best_agent = agents[best_agent_idx]

        worst_agent_idx = np.argmin(fitness)
        worst_agent = agents[worst_agent_idx]

        # Update the agents based on the behavior of players in a hide-and-seek game
        for j in range(n_agents):
            if j != best_agent_idx:
                # Calculate the distance between the current agent and the best agent
                diff_best = best_agent - agents[j]
                dist_best = np.linalg.norm(diff_best)

                # Update the agent's position towards the best agent
                factor_best = np.random.uniform()
                agents[j] += factor_best * diff_best / dist_best

                # Clip the agent's position to the search space
                agents[j] = np.clip(agents[j], lb, ub)

            if j != worst_agent_idx:
                # Calculate the distance between the current agent and the worst agent
                diff_worst = worst_agent - agents[j]
                dist_worst = np.linalg.norm(diff_worst)

                # Update the agent's position away from the worst agent
                factor_worst = np.random.uniform()
                agents[j] -= factor_worst * diff_worst / dist_worst

                # Clip the agent's position to the search space
                agents[j] = np.clip(agents[j], lb, ub)

            # Take influence from other players using roulette wheel selection
            if j != best_agent_idx and j != worst_agent_idx:
                influential_idx = roulette_wheel_selection(agents, fitness)

                # Update the agent's position based on the influential player
                agents[j] += (agents[influential_idx] - agents[j]) * np.random.uniform()

                # Clip the agent's position to the search space
                agents[j] = np.clip(agents[j], lb, ub)

        # Evaluate the new population
        new_fitness = np.array([objective_function(agent) for agent in agents])

        # Compare new fitness with old fitness and update if necessary
        for j in range(n_agents):
            if new_fitness[j] < fitness[j]:
                fitness[j] = new_fitness[j]
            else:
                # If new fitness is not better, return to the previous position
                agents[j] = old_agents[j]

        old_agents = agents.copy()

        # Print the progress if verbose mode is enabled
        if verbose:
            print(f"Iteration {i + 1}: Fitness value: {fitness.min()}")

    # Return the best solution and fitness value
    best_idx = np.argmax(fitness)
    best_solution_hogo = agents[best_idx]
    best_fitness_hogo = fitness[best_idx]

    return best_solution_hogo, best_fitness_hogo

def roulette_wheel_selection(agents, fitness):
    # Perform roulette wheel selection to determine influential player's index
    total_fitness = np.sum(fitness)
    probabilities = fitness / total_fitness
    chosen_idx = np.random.choice(len(agents), p=probabilities)
    return chosen_idx


## <span style="color:blue;">6. League Championship algorithms</span> <a class="anchor" id="lca"></a>

In [ ]:
def lca(objective_function, n_iterations, n_teams, n_players, n_dimensions, lb, ub, verbose=False):
    # Initialize the teams randomly within the search space
    teams = np.random.uniform(lb, ub, size=(n_teams, n_players, n_dimensions))

    # Evaluate the initial population
    fitness = np.array([[objective_function(player) for player in team] for team in teams])

    # Iterate over the specified number of iterations
    for i in range(n_iterations):
        # Select the best teams to advance to the next division
        best_teams_idx = np.argsort(np.sum(fitness, axis=1))[:n_teams // 2]
        best_teams = teams[best_teams_idx]

        # Update the teams based on the performance of the best teams
        for j in range(n_teams):
            if j not in best_teams_idx:
                
                # Select two random teams to compete
                team1, team2 = np.random.choice(best_teams, size=2, replace=False)

                # Calculate the fitness of each team
                fitness1 = np.sum([objective_function(player) for player in team1])
                fitness2 = np.sum([objective_function(player) for player in team2])

                # Update the losing team's players based on the winning team's players
                if fitness1 > fitness2:
                    loser, winner = team2, team1
                else:
                    loser, winner = team1, team2

                for k in range(n_players):
                    winner[k] = loser[np.random.randint(n_players)][:]
                    winner[k] += np.random.uniform(-1, 1, size=n_dimensions) * (ub - lb)

                   # Clip the player's position to the search space
                   winner[k] = np.clip(winner[k], lb, ub)

                # Evaluate the new team's fitness
                fitness[j] = np.array([objective_function(player) for player in winner])

        # Print the progress if verbose mode is enabled
        if verbose:
            print(f"Iteration {i + 1}: Fitness value: {fitness.min()}")

    # Return the best solution and fitness value
    best_team_idx = np.argmax(np.sum(fitness, axis=1))
    best_solution_lca = teams[best_team_idx]
    best_fitness_lca = np.sum(fitness[best_team_idx])

    return best_solution_lca, best_fitness_lca


## <span style="color:blue;">7. Most Valuable Player algorithms</span> <a class="anchor" id="mvpa"></a>

In [37]:
import numpy as np

def individual_competition(team, franchise_player):
    rand = np.random.uniform()
    new_team = {}
    for key, value in team.items():
        if isinstance(value, (int, float)):
            new_team[key] = value + rand * (franchise_player[key] - value) + rand * (np.random.uniform() - 0.5)
        else:
            new_team[key] = value  
    return new_team


def team_competition(team1, team2, fitness1, fitness2):
    prob_team1_wins = fitness1 / (fitness1 + fitness2)
    if np.random.rand() < prob_team1_wins:
        return team1, team2
    else:
        return team2, team1

def update_skills_after_competition(winner, loser, franchise_player):
    rand = np.random.uniform()
    new_winner, new_loser = {}, {}
    for key, value in winner.items():
        if isinstance(value, (int, float)):
            new_winner[key] = value + rand * (franchise_player[key] - value)
        else:
            new_winner[key] = value  

    for key, value in loser.items():
        if isinstance(value, (int, float)):
            new_loser[key] = value + rand * (franchise_player[key] - value)
        else:
            new_loser[key] = value 

    return new_winner, new_loser

def mutate_solution(solution, param_distributions, mutation_rate):
    mutated_solution = {key: value for key, value in solution.items()}
    for key in param_distributions.keys():
        if np.random.rand() < mutation_rate:
            mutated_solution[key] = np.random.choice(param_distributions[key])
    return mutated_solution

def mvpa(objective_function, n_solutions, param_distributions, n_iterations=10, mutation_rate=0.1):
    # Initialize the population
    solutions = [{key: np.random.choice(param_distributions[key]) for key in param_distributions} for _ in range(n_solutions)]

    # Evaluate the initial population
    fitness = np.array([objective_function(solution) for solution in solutions])

    # Iterate over the specified number of iterations
    for _ in range(n_iterations):
        
        # Identify the best solution
        best_idx = np.argmax(fitness)
        best_solution = solutions[best_idx]
        best_fitness = fitness[best_idx]

        # Individual competition for each team
        for j in range(n_solutions):
            solutions[j] = individual_competition(solutions[j], best_solution)

        # Team competition
        for j in range(n_solutions):
            if j != best_idx:
                winner, loser = team_competition(best_solution, solutions[j], best_fitness, fitness[j])
                solutions[j], best_solution = update_skills_after_competition(winner, loser, best_solution)

        # Mutation to create new solutions
        new_solutions = [mutate_solution(solution, param_distributions, mutation_rate) for solution in solutions]

        # Evaluate the new population
        new_fitness = np.array([objective_function(solution) for solution in new_solutions])

        # Replace the old population with the new one if fitness improves
        if np.max(new_fitness) > best_fitness:
            solutions, fitness = new_solutions, new_fitness
            best_idx = np.argmax(new_fitness)
            best_solution_mvpa, best_fitness_mvpa = new_solutions[best_idx], new_fitness[best_idx]

    return best_solution_mvpa, best_fitness_mvpa


In [134]:
#a simpler mvpa
def mvpa(objective_function, n_solutions, param_distributions, n_iterations=10, mutation_rate=0.1):
    # Initialize the population
    solutions = [dict(zip(param_distributions.keys(), [np.random.choice(param_distributions[key]) for key in param_distributions])) for _ in range(n_solutions)]

    # Evaluate the initial population
    fitness = np.array([objective_function(solution) for solution in solutions])

    # Iterate over the specified number of iterations
    for i in range(n_iterations):
        # Identify the best solution
        best_idx = np.argmax(fitness)
        best_solution = solutions[best_idx]
        best_fitness = fitness[best_idx]

        # Apply mutation to create new solutions
        new_solutions = []
        for solution in solutions:
            mutated_solution = {key: value for key, value in solution.items()}
            for key in param_distributions.keys():
                if np.random.rand() < mutation_rate:
                    mutated_solution[key] = np.random.choice(param_distributions[key])
            new_solutions.append(mutated_solution)

        # Evaluate the new population
        new_fitness = np.array([objective_function(solution) for solution in new_solutions])

        # Replace the old population with the new one if fitness improves
        if np.max(new_fitness) > best_fitness:
            solutions = new_solutions
            fitness = new_fitness
            best_idx = np.argmax(new_fitness)
            best_solution = new_solutions[best_idx]
            best_fitness = new_fitness[best_idx]

    return best_solution, best_fitness



## <span style="color:blue;">8. Testing</span> <a class="anchor" id="test"></a>

### 1.0. Parameter Deinition

In [29]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'poly']
}

### 1.1. TOW

In [30]:
best_solution_two, best_fitness_two = tug_of_war_optimization(objective_function, 5,2, 2, 0, 2)

Iteration 1: Fitness values: [0.955 0.97 ]
Iteration 2: Fitness values: [0.955 0.97 ]
Iteration 3: Fitness values: [0.955 0.97 ]
Iteration 4: Fitness values: [0.955 0.97 ]
Iteration 5: Fitness values: [0.955 0.97 ]


In [31]:
optimized_svm_tow = SVC(C=best_solution_two[0], gamma=best_solution_two[1], kernel='rbf')
optimized_svm_tow.fit(l_X_train, l_Y_train)

SVC(C=1.4997705889396502, gamma=0.5684740573637379)

### 1.2. HOGO

In [33]:
n_agents = 10
n_dimensions = len(param_grid)
lb = np.array([0.01] * n_dimensions)
ub = np.array([10.0] * n_dimensions)

best_solution_hogo, best_fitness_hogo = hogo(objective_function, 1, n_agents, n_dimensions, lb, ub)

In [34]:
optimized_svm_hogo = SVC(C=best_solution_hogo[0], gamma=best_solution_hogo[1], kernel='rbf')
optimized_svm_hogo.fit(l_X_train, l_Y_train)

SVC(C=7.072905856594617, gamma=0.752000978552471)

### 1.3. LCA

In [ ]:
#n_teams = 10
#n_dimensions = len(param_grid)
#lb = np.array([0.01] * n_dimensions)
#ub = np.array([2.0] * n_dimensions)

#best_solution_lca, best_fitness_lca = lca(objective_function, 100, n_teams, 1,1, 1, 2)

In [ ]:
#optimized_svm = SVC(C=best_solution_lca[0], gamma=best_solution_lca[1], kernel='rbf')
#optimized_svm.fit(l_X_train, l_Y_train)

### 1.4. MVPA

In [135]:
n_solutions = 10  # population size
best_solution, best_fitness = mvpa(objective_function, n_solutions, param_grid)

In [136]:
optimized_svm_mvpa = SVC(C=best_solution_mvpa["C"], gamma=best_solution_mvpa["gamma"], kernel='rbf', class_weight='balanced')
optimized_svm_mvpa.fit(l_X_train, l_Y_train)

SVC(C=100.0, class_weight='balanced', gamma=0.01)

## <span style="color:blue;">9. Evaluation</span> <a class="anchor" id="eval"></a>

### 1.1. TOW

In [32]:
y_pred_val = optimized_svm_tow.predict(l_X_test)
print("Classification Report for the Optimized SVM Classifier:")
accuracy = accuracy_score(l_Y_test, y_pred_val)
print(f"Best F1 score: {accuracy}")
print(classification_report(l_Y_test, y_pred_val))
print(confusion_matrix(l_Y_test, y_pred_val,))

Classification Report for the Optimized SVM Classifier:
Best F1 score: 0.97
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       100
           1       0.98      0.96      0.97       100

    accuracy                           0.97       200
   macro avg       0.97      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200

[[98  2]
 [ 4 96]]


### 1.2. HOGO

In [35]:
from sklearn.metrics import classification_report

y_pred_val = optimized_svm_hogo.predict(l_X_test)
print("Classification Report for the Optimized SVM Classifier:")
accuracy = accuracy_score(l_Y_test, y_pred_val)
print(f"Best F1 score: {accuracy}")
print(classification_report(l_Y_test, y_pred_val))
print(confusion_matrix(l_Y_test, y_pred_val,))

Classification Report for the Optimized SVM Classifier:
Best F1 score: 0.97
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       100
           1       0.98      0.96      0.97       100

    accuracy                           0.97       200
   macro avg       0.97      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200

[[98  2]
 [ 4 96]]


### 1.3. LCA

In [ ]:
#y_pred_val = optimized_svm_hogo.predict(l_X_test)
#print("Classification Report for the Optimized SVM Classifier:")
#accuracy = accuracy_score(l_Y_test, y_pred_val)
#print(f"Best F1 score: {accuracy}")
#print(classification_report(l_Y_test, y_pred_val))
#print(confusion_matrix(l_Y_test, y_pred_val,))

### 1.4. MVPA

In [137]:
y_pred_val = optimized_svm_mvpa.predict(l_X_test)
print("Classification Report for the Optimized SVM Classifier:")
accuracy = accuracy_score(l_Y_test, y_pred_val)
print(f"Best F1 score: {accuracy}")
print(classification_report(l_Y_test, y_pred_val))
print(confusion_matrix(l_Y_test, y_pred_val,))

Classification Report for the Optimized SVM Classifier:
Best F1 score: 0.97
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       100
           1       0.98      0.96      0.97       100

    accuracy                           0.97       200
   macro avg       0.97      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200

[[98  2]
 [ 4 96]]


## <span style="color:blue;">10. Ludo Game function</span> <a class="anchor" id="ludo"></a>

In [123]:
#this implementation of ludo game is a simple approach , i did not use the 4 metaheurstics instead i just used
#a placeholder function to simulate the ludo game process, that's because i didn't have all my 4 metaheuristics ready since LCA
#wasn't completely functionnal 

def ludo_game_algorithm(total_tokens, num_iterations, num_parts, X_train, Y_train, X_test, Y_test):
    # Step 1: Select the total tokens (NT) and randomly initialize in the search space
    tokens = np.random.rand(total_tokens)
    
    
    for iteration in range(num_iterations):
        # Step 2: Calculate the fitness of NT
        fitness = calculate_fitness(tokens, X_train, Y_train, X_test, Y_test)
        
        # Step 4: Determine the best token from fitness value
        best_token = tokens[np.argmax(fitness)]
        
        # Begin Intraloop
        intraloop = num_parts
        while intraloop >= 1:
            # Step 10: Calculate intraloop
            # Step 11: Select part, where (i c {1,2} for LGSI 2, ic (1,2....4}for LGSI 4)
            part = select_part(num_parts)
            
            # Step 12: Execute part, using Random Six algorithm
            token = random_six(tokens, best_token, part)
            
            # Step 13: Randomly select a token T, from part
            # Step 14: Apply corresponding algorithm of part, to update T intraloop-intraloop-1;
            tokens = update_token(tokens, token, part, X_train, Y_train, X_test, Y_test)
            
            # End Intraloop
            intraloop -= 1
        
        # Step 15: Select the best token T, in total population (NT).
        best_token = tokens[np.argmax(fitness)]
    
    # Step 16: Output T
    return best_token

def calculate_fitness(tokens, X_train, Y_train, X_test, Y_test):
    return np.random.rand(len(tokens))

def select_part(num_parts):
    # Step 11: Select part, where (i c {1,2} for LGSI 2, ic (1,2....4}for LGSI 4)
    return np.random.randint(1, num_parts + 1)

def random_six(tokens, best_token, part):
    # Step 12: Execute part, using Random Six algorithm
  
    return np.random.rand()

def update_token(tokens, token, part, X_train, Y_train, X_test, Y_test):
    # Step 14: Apply corresponding algorithm of part, to update T intraloop-intraloop-1;
    return tokens + np.random.rand(len(tokens))

# Example usage
total_tokens = 10
num_iterations = 5
num_parts = 2
X_train, Y_train, X_test, Y_test = np.random.rand(4, 10)

output_token = ludo_game_algorithm(total_tokens, num_iterations, num_parts, X_train, Y_train, X_test, Y_test)
print("Output Token:", output_token)


Output Token: 6.886751988172646
